In [ ]:
Task 2: Deep Learning on Raw Inertial Sensor Data

In [1]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, LSTM, InputLayer
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [5]:
# Base path for the dataset
DATASET_BASE_PATH = r"C:\Users\neelp\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset"


In [7]:
# Function to load raw data
def load_raw_data(folder):
    # Define the folder paths
    folder_path = os.path.join(DATASET_BASE_PATH, folder, 'Inertial Signals')
    
    # Load total acceleration
    total_acc_x = np.loadtxt(os.path.join(folder_path, f'total_acc_x_{folder}.txt'))
    total_acc_y = np.loadtxt(os.path.join(folder_path, f'total_acc_y_{folder}.txt'))
    total_acc_z = np.loadtxt(os.path.join(folder_path, f'total_acc_z_{folder}.txt'))

    # Load body acceleration
    body_acc_x = np.loadtxt(os.path.join(folder_path, f'body_acc_x_{folder}.txt'))
    body_acc_y = np.loadtxt(os.path.join(folder_path, f'body_acc_y_{folder}.txt'))
    body_acc_z = np.loadtxt(os.path.join(folder_path, f'body_acc_z_{folder}.txt'))

    # Load body gyroscope
    body_gyro_x = np.loadtxt(os.path.join(folder_path, f'body_gyro_x_{folder}.txt'))
    body_gyro_y = np.loadtxt(os.path.join(folder_path, f'body_gyro_y_{folder}.txt'))
    body_gyro_z = np.loadtxt(os.path.join(folder_path, f'body_gyro_z_{folder}.txt'))
    
    # Stack all features along the last axis
    X = np.stack([total_acc_x, total_acc_y, total_acc_z,
                  body_acc_x, body_acc_y, body_acc_z,
                  body_gyro_x, body_gyro_y, body_gyro_z], axis=-1)
    
    # Load labels
    labels_path = os.path.join(DATASET_BASE_PATH, folder, f'y_{folder}.txt')
    y = np.loadtxt(labels_path).astype(int) - 1  # Adjust labels to start from 0
    return X, y

In [9]:
# Load training and test datasets
X_train, y_train = load_raw_data('train')
X_test, y_test = load_raw_data('test')

# Verify dataset shape
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# One-hot encode labels
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Input shape
input_shape = X_train.shape[1:]

X_train shape: (7352, 128, 9)
X_test shape: (2947, 128, 9)
y_train shape: (7352,)
y_test shape: (2947,)


In [11]:

# Define 1D-CNN model
def create_cnn():
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
        Conv1D(64, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Define MLP model
def create_mlp():
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
# Define LSTM model
def create_lstm():
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, return_sequences=True),
        LSTM(64),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# Function to train and evaluate models
def train_and_evaluate(model, model_name):
    print(f"Training {model_name}...")
    history = model.fit(X_train, y_train_categorical, validation_split=0.2, epochs=20, batch_size=32, verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    
    print(f"\n--- {model_name} Performance ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(classification_report(y_test, y_pred_classes))
    print("="*50)
    return history

In [19]:

# Train and evaluate each model
cnn_model = create_cnn()
cnn_history = train_and_evaluate(cnn_model, "1D-CNN")

mlp_model = create_mlp()
mlp_history = train_and_evaluate(mlp_model, "MLP")

lstm_model = create_lstm()
lstm_history = train_and_evaluate(lstm_model, "LSTM")


Training 1D-CNN...
Epoch 1/20


C:\Users\neelp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6694 - loss: 0.7826 - val_accuracy: 0.8790 - val_loss: 0.4601
Epoch 2/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9193 - loss: 0.2217 - val_accuracy: 0.9055 - val_loss: 0.3810
Epoch 3/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9505 - loss: 0.1295 - val_accuracy: 0.9001 - val_loss: 0.4312
Epoch 4/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9496 - loss: 0.1182 - val_accuracy: 0.9109 - val_loss: 0.4702
Epoch 5/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9564 - loss: 0.0963 - val_accuracy: 0.9075 - val_loss: 0.3936
Epoch 6/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9488 - loss: 0.1148 - val_accuracy: 0.8817 - val_loss: 0.7350
Epoch 7/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9555 - loss: 0.1002 - val_accuracy: 0.9232 - val_loss: 0.3854
Epoch 8/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9547 - loss: 0.0959 - val_accuracy: 0.9143 - val

C:\Users\neelp\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5778 - loss: 1.0961 - val_accuracy: 0.8681 - val_loss: 0.5040
Epoch 2/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8568 - loss: 0.4137 - val_accuracy: 0.8749 - val_loss: 0.4130
Epoch 3/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9046 - loss: 0.2593 - val_accuracy: 0.8776 - val_loss: 0.4250
Epoch 4/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9239 - loss: 0.2000 - val_accuracy: 0.8980 - val_loss: 0.4217
Epoch 5/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9399 - loss: 0.1504 - val_accuracy: 0.8919 - val_loss: 0.4646
Epoch 6/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9421 - loss: 0.1462 - val_accuracy: 0.8960 - val_loss: 0.5006
Epoch 7/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9489 - loss: 0.1306 - val_accuracy: 0.8987 - val_loss: 0.5996
Epoch 8/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9543 - loss: 0.1155 - val_accuracy: 0.9035 - val_

C:\Users\neelp\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


184/184 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.4457 - loss: 1.3201 - val_accuracy: 0.6037 - val_loss: 0.9872
Epoch 2/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.6594 - loss: 0.7572 - val_accuracy: 0.7356 - val_loss: 0.7783
Epoch 3/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.7916 - loss: 0.5482 - val_accuracy: 0.7213 - val_loss: 0.7399
Epoch 4/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - accuracy: 0.8337 - loss: 0.4602 - val_accuracy: 0.8620 - val_loss: 0.6195
Epoch 5/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.9050 - loss: 0.2855 - val_accuracy: 0.8804 - val_loss: 0.5607
Epoch 6/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.9357 - loss: 0.1927 - val_accuracy: 0.8872 - val_loss: 0.4877
Epoch 7/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.9425 - loss: 0.1551 - val_accuracy: 0.8402 - val_loss: 0.5956
Epoch 8/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9379 - loss: 0.1651 - val_accurac

In [ ]:
Task 2: Deep Learning Models
Objective: Train and evaluate deep learning models (1D-CNN, MLP, and LSTM) using the raw inertial sensor data from the UCI HAR dataset.
Approach:
Preprocessed the raw data and applied various deep learning architectures:
1D-CNN: Achieved 90% accuracy with excellent performance across all activity classes.
MLP: Achieved 88.6% accuracy, with slightly lower performance compared to CNN.
LSTM: Achieved 91.4% accuracy, showing strong performance with sequential data.
Results:
Best Model: 1D-CNN (90.4% accuracy, 0.90 F1 score, demonstrating strong performance in sequential and time-series data).